<a href="https://colab.research.google.com/github/rdemarqui/llm_complaint_management/blob/main/02.%20LLM_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM at Work: Decoding Customer Feedback

This study aims to explore these technological advancements, specifically using the advanced LLM tool Mistral 7B, to analyze a vast collection of complaints gathered from the website [reclameaqui.com.br](reclameaqui.com.br). Through this analysis, we intend to demonstrate the potential and effectiveness of LLMs in interpreting and handling large-scale user feedback.

For more information, please visit my github page: https://github.com/rdemarqui/llm_complaint_management

## Complaint Dataset

### Load

In [1]:
from IPython.display import clear_output
import pandas as pd

In [9]:
# Loading data
data_url = 'https://raw.githubusercontent.com/rdemarqui/llm_complaint_management/main/datasets/full_dataset_claro.xlsx'
df = pd.read_excel(data_url, usecols=['problem', 'description'])
print(df.shape)

(7000, 2)


### Clean

In [8]:
# Remove extra space and uncase all words
df['description'] = df['description'].str.replace(r'\s{2,}', ' ', regex=True)
df['description'] = df['description'].str.lower()
df.head()

,problem,description
0,Cobrança indevida,recebi um e-mail spam com uma fatura no valor ...
1,Cobrança indevida,"compra não autoriza na meu cartão, chip de núm..."
2,Cobrança indevida,no dia 11 de outubro realizei uma recarga pelo...
3,Cobrança indevida,segue o número de protocolo do atendimento rea...
4,Cobrança indevida,fiz hoje dia 12/10/2023 uma recarga de celular...


### Analysis

## Text Classification

### Loading Model

There are several 7B models available as open source, such as Mistral, Falcon, Zephyr, and Openchat. In this study, we will use Mistral, but feel free to test with other models, paying attention only to the instruction structure.

In [ ]:
import time
from datetime import timedelta
from google.colab import files
from IPython.display import clear_output

In [ ]:
# Latest HF transformers version for Mistral-like models
!pip install git+https://github.com/huggingface/transformers.git

# GPTQ Dependencies
!pip install optimum #transformers
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

clear_output()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
# Mistral revisions
# main, gptq-4bit-32g-actorder_True
# gptq-8bit--1g-actorder_True
# gptq-8bit-128g-actorder_True
# gptq-8bit-32g-actorder_True,
# gptq-4bit-64g-actorder_True

In [ ]:
# Load LLM and Tokenizer
model_id = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
    #revision="gptq-4bit-32g-actorder_True"
)
clear_output()

In [ ]:
# Create a pipeline
generator = pipeline(model=model,
                tokenizer=tokenizer,
                task='text-generation',
                max_new_tokens=50,
                temperature=0.1,
                do_sample=True
                )
clear_output()

### Prompt Engineering

Prompt Guide:
* https://www.promptingguide.ai/models/mistral-7b

In [ ]:
# This is the footprint of your prompt. This will be replicated in all prompts
footprint_prompt = """
[INST]
Classifique a reclamação baseado em uma ou mais categorias abaixo:
sinal/conexão de rede; cobrança indevida; consumo saldo/crédito; plano/benefício; furto/roubo; cancelamento; recuperar número/linha; chip/sim card; spam; portabilidade.

Atenção use somente as categorias, não explique!

Reclamação: {user_complain}
[/INST]"""

In [ ]:
# Example prompt
compl = "O meu celular está sempre sem barrinha de rede. Não pega em nunhum lugar!"
reason = " sinal/conexão de rede"
example_prompt = "<s>" + footprint_prompt.format(user_complain=compl) + reason + "</s>"

In [ ]:
#prompt = footprint_prompt # zero shot
prompt = example_prompt + footprint_prompt # one shot
print(prompt)

<s>
[INST]
Classifique a reclamação baseado em uma ou mais categorias abaixo:
sinal/conexão de rede; cobrança indevida; consumo saldo/crédito; plano/benefício; furto/roubo; cancelamento; recuperar número/linha; chip/sim card; spam; portabilidade.

Atenção use somente as categorias, não explique!

Reclamação: O meu celular está sempre sem barrinha de rede. Não pega em nunhum lugar!
[/INST] sinal/conexão de rede</s>
[INST]
Classifique a reclamação baseado em uma ou mais categorias abaixo:
sinal/conexão de rede; cobrança indevida; consumo saldo/crédito; plano/benefício; furto/roubo; cancelamento; recuperar número/linha; chip/sim card; spam; portabilidade.

Atenção use somente as categorias, não explique!

Reclamação: {user_complain}
[/INST]


In [ ]:
def llm_analysis(complain):
  response = generator(prompt.format(user_complain=complain))
  llm_classification = response[0]['generated_text'].split('[/INST]')[-1][1:].replace('.', '')

  return llm_classification

In [ ]:
# Example
complain ="""
Paguei a fatura na data correta e estou sendo indevidamente cobrado novamente pela mesma como se não tivesse pago e o que é pior cortaram meu sinal.
""".lower()+'\n'

llm_analysis(complain)

### Classification

In [ ]:
# Play an audio beep when the anaysis is complete
from google.colab import output

https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0

F12 para abrir o inspetor, vá até a aba console, cole o código abaixo e pressione enter.

`function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)`

In [ ]:
def iterate_over_interval(df, column, limit_char, start_row, end_row, save_interval):
  if start_row < 0 or end_row >= len(df):
    raise ValueError(f"Intervalo de linhas inválido. Limite do dataset: {len(df)-1}")

  line_count = 0

  for index in range(start_row, end_row + 1):
    complaint_content = str(df.loc[index, column]).lower()

    # Limit text lengh
    if len(complaint_content) > limit_char: complaint_content = complaint_content[:limit_char]+'\n'

    analysis = llm_analysis(complaint_content)
    df.at[index, 'classificação'] = analysis

    line_count += 1

    # Save partial analisys
    if line_count % save_interval == 0:
      file_save = 'analise_reclamacao_' + str(start_row) + '_' + str(end_row) + '_' + str(index) + '.xlsx'
      print(f"Analise numero {line_count}. Salvando arquivo: {file_save}")
      df_analisys = df[['Protocolo', 'classificação']]
      df_analisys.to_excel(file_save, index=False)
      #files.download(file_save)
      output.eval_js('new Audio("https://raw.githubusercontent.com/rdemarqui/studies/master/Metal-Gear-Solid-Alert-_Sound-Effect_.ogg").play()')

  # Save final result
  file_save = 'analise_reclamacao_' + str(start_row) + '_' + str(end_row) + '_' + str(index) + '.xlsx'
  print(f"Analise numero {line_count}. Salvando arquivo: {file_save}")
  df_analisys = df[['Protocolo', 'classificação']]
  df_analisys.to_excel(file_save, index=False)
  output.eval_js('new Audio("https://raw.githubusercontent.com/rdemarqui/studies/master/Metal-Gear-Solid-Alert-_Sound-Effect_.ogg").play()')

In [ ]:
start_time = time.time()

iterate_over_interval(df=df, column="Conteúdo Reclamação",
                      limit_char=1700,
                      start_row=457, end_row=3230,
                      save_interval=500)

end_time = time.time()
delta_time = str(timedelta(seconds=end_time - start_time))
print(f"Execution: {delta_time}")

Analise numero 500. Salvando arquivo: analise_reclamacao_457_3230_956.xlsx
Analise numero 1000. Salvando arquivo: analise_reclamacao_457_3230_1456.xlsx
Analise numero 1500. Salvando arquivo: analise_reclamacao_457_3230_1956.xlsx
Analise numero 2000. Salvando arquivo: analise_reclamacao_457_3230_2456.xlsx
Analise numero 2500. Salvando arquivo: analise_reclamacao_457_3230_2956.xlsx
Analise numero 2774. Salvando arquivo: analise_reclamacao_457_3230_3230.xlsx
Execution: 1:26:46.831932


## Results

In [ ]:
df

---